In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Activation, MaxPooling2D, BatchNormalization, Dropout

num_classes = 20
size = 32
batch_size = 16

train_dir = "./data/train"
test_dir = "./data/validation"

train_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_data = ImageDataGenerator(rescale=1./255)

train_generator = train_data.flow_from_directory(
    train_dir,
    target_size=(size, size),
    class_mode="categorical",
    batch_size=batch_size,
)

test_generator = test_data.flow_from_directory(
    test_dir,
    target_size=(size, size),
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False
)

Using TensorFlow backend.
Found 19548 images belonging to 20 classes.
Found 990 images belonging to 20 classes.


In [3]:
model = Sequential()

model.add(Conv2D(64, (3,3), padding="same", input_shape=(size,size,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.2))

model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.2))

model.add(Conv2D(256, (3,3), padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.2))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

model.add(Dense(num_classes, activation="softmax"))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)       

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(
    "./simpsons.v1.h5", 
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    verbose=1)

earlystop = EarlyStopping(monitor="val_accuracy",min_delta=0, patience=3, verbose=1, restore_best_weights=True)

callbacks = [checkpoint, earlystop]

model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=['accuracy'])

nb_train = 19548
nb_test = 990

epochs = 10
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train//batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks,
    validation_data=test_generator,
    validation_steps=nb_test//batch_size,
)

Epoch 1/10
1221/1221 [==============================] - 63s 51ms/step - loss: 3.0763 - accuracy: 0.1560 - val_loss: 3.1414 - val_accuracy: 0.2305

Epoch 00001: val_accuracy improved from -inf to 0.23053, saving model to ./simpsons.v1.h5
Epoch 2/10
1221/1221 [==============================] - 58s 47ms/step - loss: 2.1886 - accuracy: 0.3407 - val_loss: 1.8615 - val_accuracy: 0.3255

Epoch 00002: val_accuracy improved from 0.23053 to 0.32546, saving model to ./simpsons.v1.h5
Epoch 3/10
1221/1221 [==============================] - 59s 48ms/step - loss: 1.7234 - accuracy: 0.4775 - val_loss: 1.4524 - val_accuracy: 0.5246

Epoch 00003: val_accuracy improved from 0.32546 to 0.52464, saving model to ./simpsons.v1.h5
Epoch 4/10
1221/1221 [==============================] - 61s 50ms/step - loss: 1.4156 - accuracy: 0.5725 - val_loss: 0.5153 - val_accuracy: 0.6057

Epoch 00004: val_accuracy improved from 0.52464 to 0.60575, saving model to ./simpsons.v1.h5
Epoch 5/10
1221/1221 [=====================